In [4]:
import configparser
import os
from pyspark.sql import SparkSession

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')
output_data = config.get('S3','OUTPUT')

In [5]:
def create_spark_session():
    """Create a spark session
    
        Args:
            - None
        
        Return:
            - An instance of SparkSession
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [6]:
spark = create_spark_session()

In [7]:
ratings  = spark.read.parquet(os.path.join(output_data,"ratings"))

In [8]:
movies = spark.read.parquet(os.path.join(output_data,"movies"))

Register temp tables so that we can use spark sql to query the tables

In [9]:
ratings.registerTempTable("ratings_table");movies.registerTempTable("movies_table")

Show some top rated movies

In [18]:
spark.sql("""
SELECT
        m.title, AVG(r.rating) AS rating
    FROM
        movies_table m
    JOIN
        ratings_table r
    ON
        m.movie_id = r.movie_id
    GROUP BY
        m.title
    HAVING
        AVG(r.rating) > 4.5
    ORDER BY
        rating DESC
""").show()

+--------------------+-----------------+
|               title|           rating|
+--------------------+-----------------+
|Song of Freedom (...|              5.0|
|Ulysses (Ulisse) ...|              5.0|
|Gate of Heavenly ...|              5.0|
|Bittersweet Motel...|              5.0|
|    Baby, The (1973)|              5.0|
|Smashing Time (1967)|              5.0|
|Follow the Bitch ...|              5.0|
|Schlafes Bruder (...|              5.0|
|        Lured (1947)|              5.0|
|One Little Indian...|              5.0|
|I Am Cuba (Soy Cu...|              4.8|
|     Lamerica (1994)|             4.75|
|Apple, The (Sib) ...|4.666666666666667|
|      Sanjuro (1962)|4.608695652173913|
|Seven Samurai (Th...|4.560509554140127|
|Shawshank Redempt...|4.554557700942973|
|Godfather, The (1...|4.524966261808367|
|Close Shave, A (1...| 4.52054794520548|
|Usual Suspects, T...|4.517106001121705|
|Schindler's List ...|4.510416666666667|
+--------------------+-----------------+
only showing top